In [ ]:
import requests
import json
import pandas as pd
import time
import timeit
import math

## Restaurant ID 2

In [ ]:
def partition(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i : i+size]

In [ ]:
get_restaurant_id_header={'accept': 'application/json, text/plain, */*'
,'x-foody-client-id': 'fa42efc4-7303-4cd8-b71f-93359fb6d50a'
,'x-foody-client-type': '4'
,'x-foody-app-type': '1004'
,'x-foody-client-version': '5.23.0'
,'x-foody-api-version': '1'
,'x-foody-client-language': 'en'
# ,'x-foody-access-token': 
,'user-agent': 'NOW/5.13.2 (sdk_gphone64_x86_64; android 33; Scale/2.75)'
,'x-foody-client-rn-version': '5.13.2'
# ,'x-foody-device-fingerprint': 
,'content-type': 'application/json; charset=UTF-8'
,'content-length': '155'
,'accept-encoding': 'gzip'}


In [ ]:
def get_browsing_infos(url,header_param, payload_param):
  header=header_param.copy()
  payload=payload_param.copy()
  header['content-length']=str(len(json.dumps(payload, separators=(',',':'))))
  respone=requests.post(url,headers=header,json=payload)
  data=json.loads(respone.text)
  return data['reply']['delivery_infos']

In [ ]:
def get_restaurant(browsing_ids,location):
    url = 'https://gappapi.deliverynow.vn/api/delivery/get_browsing_infos'
    browsing_ids_p = list(partition(browsing_ids, 10))
    browsing_infos=[]
    for browsing_ids_10 in browsing_ids_p:
      get_restaurant_id_payload={
                            "city_id": 217,
                            "delivery_ids": browsing_ids_10,
                            "position": {
                                "latitude": location[0],
                                "longitude": location[1]
                              }
                            }
      browsing_infos= browsing_infos + get_browsing_infos(url,get_restaurant_id_header,get_restaurant_id_payload)
    return browsing_infos


In [ ]:
def get(browsing_ids,location): # lấy location_url --> loại các điểm không thuộc tp HCM
 # thiếu district_id --> đếm số lượng quán tại quận, huyện
  browsing_infos = get_restaurant(browsing_ids,location)
  browsing_id=[];restaurant_id=[]; restaurant_url=[];restaurant_name=[];is_quality_merchant=[];
  restaurant_status=[];limit_distance=[]; categories=[];cuisines=[]
  brand_id=[];brand_name=[]; address=[];lon=[];lat=[]
  status=[];open_time=[];close_time=[]
  location_url = [];district_id=[]

  for info in browsing_infos:
    browsing_id.append(info['id'])
    restaurant_id.append(info['restaurant_id'])
    restaurant_url.append(info['restaurant_url'])
    restaurant_name.append(info['name'])
    is_quality_merchant.append(info['is_quality_merchant'])
    restaurant_status.append(info['restaurant_status'])
    limit_distance.append(info['limit_distance'])
    categories.append('*#*'.join(info['categories'])); cuisines.append('*#*'.join(info['cuisines']))
    location_url.append(info['location_url']); district_id.append(info['district_id'])

  
    if 'brand' in info:
      brand_id.append(info['brand']['brand_id'])
      brand_name.append(info['brand']['name'])
    else:
      brand_id.append(None)
      brand_name.append(None)

    address.append(info['address']); lon.append(info['position']['longitude']); lat.append(info['position']['latitude'])
    if 'operating' in info:
        if 'status' in info['operating']:
          status.append(info['operating']['status'])
        else:
          status.append(None)
        if 'open_time' in info['operating']:
          open_time.append(info['operating']['open_time'])
        else:
          open_time.append(None)
        if 'close_time' in info['operating']:
          close_time.append(info['operating']['close_time'])
        else:
          close_time.append(None)
  data_dict={
  'district_id':district_id, 'location_url':location_url,
  'browsing_id':browsing_id,'restaurant_id':restaurant_id, 'restaurant_url':restaurant_url,
  'restaurant_name':restaurant_name,'is_quality_merchant':is_quality_merchant,
  'restaurant_status':restaurant_status,'limit_distance':limit_distance,
  'categories':categories,'cuisines':cuisines,
  'brand_id':brand_id,'brand_name':brand_name,
  'address':address,'lon':lon,'lat':lat,
  'status':status,'open_time':open_time,'close_time':close_time  
  }
  return pd.DataFrame(data_dict)

In [ ]:
ids_df = pd.read_excel("/content/ids.xlsx")
locations_df = pd.read_excel("/content/locations.xlsx")
locations_df

In [ ]:
list_df=[]
for i in locations_df.index:
  location_ = [locations_df.lat[i],locations_df.lon[i]]
  df_gr_lat = ids_df[ids_df.lat ==location_[0]]
  df_lat_lon = df_gr_lat[df_gr_lat.lon ==location_[1]]
  l_ids = df_lat_lon['id'].tolist()
  list_df.append(get(l_ids,location_))

full_df = pd.concat(list_df)
full_df

In [ ]:
full_df.to_excel('out.xlsx') # chưa loại tỉnh thành